In [21]:
import pandas as pd 
import numpy as np
from scapy.all import * 
from datetime import datetime
from datetime import UTC 
from collections import defaultdict

# Creating Main DataFrame 

Setting TCP Flag Mappings

In [22]:
# TCP Flags Mapping
# Check here for bitmap: https://www.noction.com/blog/tcp-flags#:~:text=The%20hexadecimal%20number%200x02%20tells,a%20particular%20flag%20is%20set.
FIN = 0x01
SYN = 0x02
RST = 0x04
PSH = 0x08
ACK = 0x10
URG = 0x20
ECE = 0x40
CWR = 0x80

Here, we make important assumption that the first flow we see in the pcap for a unique flow_id, is considered as the "forward packet". This may not necessarily be the packet initiated by the host (perhaps), but it is a fair assumption to make. 

This assumption just helps us define a direction of the flow and should not change anything major. If any flow has a lot of packets being sent from one direction (think DDoS), then this imbalance will be captured in the fwd or bwd total packets column. (should not matter which one specifically) 

In [23]:
def create_dataframe(INPUT_FILE):

    """Finds all unique flows based on flow id. Returns data frame with basic metrics computed for each flow. """
    print("Reading .pcap file.")
    packets = rdpcap(INPUT_FILE)
    print("Reading .pcap file DONE.")


    print("Creating initial dataframe.")
    all_data = {}
    flow_fwd_states = []
    i = 0
    for pkt in packets:
    
        if IP in pkt:
            tmp_pack_dict = {}

            tmp_pack_dict["sport"] = pkt[IP].sport if hasattr(pkt[IP], "sport") else None
            tmp_pack_dict["src_ip"] = pkt[IP].src 
            tmp_pack_dict["dst_port"] = pkt[IP].dport if hasattr(pkt[IP], "dport") else None
            tmp_pack_dict["dst_ip"] = pkt[IP].dst
            flow_size = pkt.len

            # Check https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml for Assigned Internet Protocol Numbers
            tmp_pack_dict['protocol'] = pkt.proto

            # Flow Unique Identifier / Flow ID 
            flow_id = frozenset([tmp_pack_dict["sport"], tmp_pack_dict["src_ip"], tmp_pack_dict["dst_port"], tmp_pack_dict["dst_ip"]])
            # Need set representation because if there is a backward flow (with just order changed of source and destination) then it should be marked as "seen" previously 
            # Ordered flow id (to check if belongs to the same stream or not)
            flow_id_ordered = (tmp_pack_dict['sport'], tmp_pack_dict['src_ip'], tmp_pack_dict['dst_port'], tmp_pack_dict['dst_ip']) # save it in order

            if flow_id not in all_data: #meaning this is a new flow (from a different stream) 
                tmp_pack_dict["sizes"] = [flow_size]
                tmp_pack_dict["first_timestamp"] = pkt.time
                tmp_pack_dict["last_timestamp"] = pkt.time 
                tmp_pack_dict["flow_duration"] = 0 
                tmp_pack_dict["arrival_times"] = [pkt.time]


                # Forward packets 
                tmp_pack_dict["total_fwd_packets"] = 1 # To count the first instance 
                tmp_pack_dict["fwd_pkt_sizes"] = [pkt.len]
                tmp_pack_dict["first_timestamp_fwd"] = pkt.time
                tmp_pack_dict["last_timestamp_fwd"] = pkt.time 
                tmp_pack_dict["arrival_times_fwd"] = [pkt.time]


                # Backward packets 
                tmp_pack_dict["total_bwd_packets"] = 0 
                tmp_pack_dict["bwd_pkt_sizes"] = [] 
                tmp_pack_dict["first_timestamp_bwd"] = -1
                tmp_pack_dict["last_timestamp_bwd"] = -1
                tmp_pack_dict["arrival_times_bwd"] = []

                # Add flag counts 
                tmp_pack_dict["syn_flag_count"] = 0
                tmp_pack_dict["fin_flag_count"] = 0
                tmp_pack_dict["rst_flag_count"] = 0 
                tmp_pack_dict["psh_flag_count"] = 0
                tmp_pack_dict["ack_flag_count"] = 0 
                tmp_pack_dict["urg_flag_count"] = 0 
                tmp_pack_dict["cwr_flag_count"] = 0
                tmp_pack_dict["ece_flag_count"] = 0

                # create first time dictionary 
                all_data[flow_id] = tmp_pack_dict
                # save the first instance of the flow as the forward trace 
                flow_fwd_states.append(flow_id_ordered)

            else: # meaning either forward or backward trace (from the same flow!)

                # Update the general features first 
                all_data[flow_id]["sizes"].append(flow_size) 
                all_data[flow_id]["first_timestamp"] = min(all_data[flow_id]["first_timestamp"], pkt.time)
                all_data[flow_id]["last_timestamp"] = max(all_data[flow_id]["last_timestamp"], pkt.time)
                all_data[flow_id]["flow_duration"] = all_data[flow_id]["last_timestamp"] - all_data[flow_id]["first_timestamp"]
                all_data[flow_id]["arrival_times"].append(pkt.time)
                

                # Add forward packet features 
                if flow_id_ordered in flow_fwd_states: # check if forward packet and not backward 
                    all_data[flow_id]["total_fwd_packets"] += 1 
                    all_data[flow_id]["fwd_pkt_sizes"].append(pkt.len) 
                    all_data[flow_id]["first_timestamp_fwd"] = min(all_data[flow_id]["first_timestamp_fwd"], pkt.time)
                    all_data[flow_id]["last_timestamp_fwd"] = max(all_data[flow_id]["last_timestamp_fwd"], pkt.time)
                    all_data[flow_id]["arrival_times_fwd"].append(pkt.time)

                else:
                    all_data[flow_id]["total_bwd_packets"] += 1 
                    all_data[flow_id]["bwd_pkt_sizes"].append(pkt.len) 
                    all_data[flow_id]["first_timestamp_bwd"] = pkt.time if all_data[flow_id]["first_timestamp_bwd"] == -1 else min(all_data[flow_id]["first_timestamp_bwd"], pkt.time)
                    all_data[flow_id]["last_timestamp_bwd"] = max(all_data[flow_id]["last_timestamp_bwd"], pkt.time)
                    all_data[flow_id]["arrival_times_bwd"].append(pkt.time)

            if TCP in pkt[IP]:
                all_data[flow_id]["syn_flag_count"] += 1 if pkt[IP][TCP].flags & SYN else 0 
                all_data[flow_id]["fin_flag_count"] += 1 if pkt[IP][TCP].flags & FIN else 0
                all_data[flow_id]["rst_flag_count"] += 1 if pkt[IP][TCP].flags & RST else 0 
                all_data[flow_id]["psh_flag_count"] += 1 if pkt[IP][TCP].flags & PSH else 0
                all_data[flow_id]["ack_flag_count"] += 1 if pkt[IP][TCP].flags & ACK else 0 
                all_data[flow_id]["urg_flag_count"] += 1 if pkt[IP][TCP].flags & URG else 0 
                all_data[flow_id]["cwr_flag_count"] += 1 if pkt[IP][TCP].flags & CWR else 0
                all_data[flow_id]["ece_flag_count"] += 1 if pkt[IP][TCP].flags & ECE else 0
                

    df = pd.DataFrame.from_dict(all_data, orient="index")
    df.reset_index(drop=True, inplace=True)

    print("Initial data frame created.")

    return df 

In [24]:
FILE = "Port_Scan_10per_streams.pcap"
df = create_dataframe(INPUT_FILE=FILE)

Reading .pcap file.
Reading .pcap file DONE.
Creating initial dataframe.
Initial data frame created.


In [25]:
df.head()

,sport,src_ip,dst_port,dst_ip,protocol,sizes,first_timestamp,last_timestamp,flow_duration,arrival_times,...,last_timestamp_bwd,arrival_times_bwd,syn_flag_count,fin_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwr_flag_count,ece_flag_count
0,40556,172.16.0.1,80,192.168.10.50,6,"[60, 60, 52, 399, 52, 416, 52, 408, 1100, 52, ...",1499441877.184978,1499441882.213039,5.028061,"[1499441877.184978, 1499441877.185112, 1499441...",...,1499441882.213039,"[1499441877.185112, 1499441877.186098, 1499441...",2,2,0,4,12,0,0,0
1,55000,172.16.0.1,443,192.168.10.50,6,"[44, 40]",1499446536.020843,1499446536.020932,0.000089,"[1499446536.020843, 1499446536.020932]",...,1499446536.020932,[1499446536.020932],1,0,1,0,1,0,0,0
2,63214,172.16.0.1,21,192.168.10.50,6,"[44, 44, 40]",1499446540.456491,1499446540.457139,0.000648,"[1499446540.456491, 1499446540.45656, 14994465...",...,1499446540.45656,[1499446540.45656],2,0,1,0,1,0,0,0
3,63214,172.16.0.1,444,192.168.10.50,6,"[44, 40]",1499446540.863161,1499446540.863282,0.000121,"[1499446540.863161, 1499446540.863282]",...,1499446540.863282,[1499446540.863282],1,0,1,0,1,0,0,0
4,38040,172.16.0.1,80,192.168.10.50,6,"[44, 44, 40]",1499446552.413619,1499446552.414284,0.000665,"[1499446552.413619, 1499446552.413746, 1499446...",...,1499446552.413746,[1499446552.413746],2,0,1,0,1,0,0,0


Running sanity check to make sure there are no duplicates based on flow_id

In [26]:
subset_cols = ["sport", "src_ip", "dst_port", "dst_ip"]
duplicates = df.duplicated(subset=subset_cols)

duplicate_rows = df[duplicates]
assert duplicate_rows.shape[0] == 0

In [27]:
duplicate_rows.shape[0]

0

In [28]:
df.columns

Index(['sport', 'src_ip', 'dst_port', 'dst_ip', 'protocol', 'sizes',
       'first_timestamp', 'last_timestamp', 'flow_duration', 'arrival_times',
       'total_fwd_packets', 'fwd_pkt_sizes', 'first_timestamp_fwd',
       'last_timestamp_fwd', 'arrival_times_fwd', 'total_bwd_packets',
       'bwd_pkt_sizes', 'first_timestamp_bwd', 'last_timestamp_bwd',
       'arrival_times_bwd', 'syn_flag_count', 'fin_flag_count',
       'rst_flag_count', 'psh_flag_count', 'ack_flag_count', 'urg_flag_count',
       'cwr_flag_count', 'ece_flag_count'],
      dtype='object')

Some sanity checks

In [9]:
# check if any first_timestamp_fwd == -1
first_fwd_timestamp = df[df.loc[:, "first_timestamp_fwd"] == -1]
assert first_fwd_timestamp.shape[0] == 0

change to NaN?

## Cleanup

### Size features

In [29]:
## Overall
df["total_size"] = round(df.loc[:, "sizes"].apply(lambda x: np.sum(x)), 3)
df["avg_size"] = round(df.loc[:, "sizes"].apply(lambda x: np.mean(x)), 3)
df["std_size"] = round(df.loc[:, "sizes"].apply(lambda x: np.std(x)), 3)

## Forward
df["total_fwd_pkt_size"] = round(df.loc[:, "fwd_pkt_sizes"].apply(lambda x: np.sum(x)), 3)
df["avg_fwd_pkt_size"] = round(df.loc[:, "fwd_pkt_sizes"].apply(lambda x: np.mean(x)), 3)
df["std_fwd_pkt_size"] = round(df.loc[:, "fwd_pkt_sizes"].apply(lambda x: np.std(x)), 3)
## Backward
df["total_bwd_pkt_size"] = round(df.loc[:, "bwd_pkt_sizes"].apply(lambda x: np.sum(x)), 3)
df["avg_bwd_pkt_size"] = round(df.loc[:, "bwd_pkt_sizes"].apply(lambda x: np.mean(x)), 3)
df["std_bwd_pkt_size"] = round(df.loc[:, "bwd_pkt_sizes"].apply(lambda x: np.std(x)),3)


/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [30]:
# check cases when first_timestamp_bwd == -1 => last_timestamp_bwd has to be -1 as well. Also, we should perhaps change total_bwd_pkt_size to NaN in this case too!
first_bwd_timestamp = df[df.loc[:, "first_timestamp_bwd"] == -1]
first_bwd_timestamp.loc[:, ["first_timestamp_bwd", "last_timestamp_bwd", "arrival_times_bwd", "total_bwd_pkt_size", "avg_bwd_pkt_size", "std_bwd_pkt_size"]]

,first_timestamp_bwd,last_timestamp_bwd,arrival_times_bwd,total_bwd_pkt_size,avg_bwd_pkt_size,std_bwd_pkt_size
15166,-1,-1,[],0.0,NaN,NaN
15167,-1,-1,[],0.0,NaN,NaN
15168,-1,-1,[],0.0,NaN,NaN
15169,-1,-1,[],0.0,NaN,NaN
15170,-1,-1,[],0.0,NaN,NaN
15171,-1,-1,[],0.0,NaN,NaN
15172,-1,-1,[],0.0,NaN,NaN
15173,-1,-1,[],0.0,NaN,NaN
15174,-1,-1,[],0.0,NaN,NaN
15175,-1,-1,[],0.0,NaN,NaN


### BWD timestamps conversion

Convert -1 values to np.nan for better readability when converting to human-readable formats ahead

In [31]:
bwd_cols = ["first_timestamp_bwd", "last_timestamp_bwd"]
df[bwd_cols] = df[bwd_cols].replace(-1, np.nan)

In [32]:
df.eq(-1).any()

sport                  False
src_ip                 False
dst_port               False
dst_ip                 False
protocol               False
sizes                  False
first_timestamp        False
last_timestamp         False
flow_duration          False
arrival_times          False
total_fwd_packets      False
fwd_pkt_sizes          False
first_timestamp_fwd    False
last_timestamp_fwd     False
arrival_times_fwd      False
total_bwd_packets      False
bwd_pkt_sizes          False
first_timestamp_bwd    False
last_timestamp_bwd     False
arrival_times_bwd      False
syn_flag_count         False
fin_flag_count         False
rst_flag_count         False
psh_flag_count         False
ack_flag_count         False
urg_flag_count         False
cwr_flag_count         False
ece_flag_count         False
total_size             False
avg_size               False
std_size               False
total_fwd_pkt_size     False
avg_fwd_pkt_size       False
std_fwd_pkt_size       False
total_bwd_pkt_

### Time-based Features

Computing flow durations

NOTE: Flow durations are in SECONDS

In [35]:
df["fwd_flow_duration"] = df.loc[:, "last_timestamp_fwd"] - df.loc[:, "first_timestamp_fwd"]
df["bwd_flow_duration"] = df.loc[:, "last_timestamp_bwd"] - df.loc[:, "first_timestamp_bwd"]
df["flow_duration"] = df.loc[:, "last_timestamp"] - df.loc[:, "first_timestamp"]
df.loc[:, ["first_timestamp", "last_timestamp", "flow_duration"]]

,first_timestamp,last_timestamp,flow_duration
0,1499441877.184978,1499441882.213039,5.028061
1,1499446536.020843,1499446536.020932,0.000089
2,1499446540.456491,1499446540.457139,0.000648
3,1499446540.863161,1499446540.863282,0.000121
4,1499446552.413619,1499446552.414284,0.000665
...,...,...,...
16085,1499451832.021263,1499451832.021281,0.000018
16086,1499451832.021495,1499451832.021539,0.000044
16087,1499451832.021606,1499451832.021649,0.000043
16088,1499451832.021759,1499451832.021813,0.000054


Check what happens to bwd_flow_duration when timestamps were -1.
Sanity check to make sure it is also NaN to differentiate from case where there was exactly one bwd packet!

In [ ]:
assert df[df.loc[:, "first_timestamp_bwd"].isna()].loc[:, "bwd_flow_duration"].isna().all() == True

NOTE: flow_duration will be 0 if only one packet was sent (overall, fwd or bwd)

In [ ]:
# Case when fwd_flow_duration is 0. total_fwd_packets should be 1 
zero_fwd_flow_duration = df[df.loc[:, "fwd_flow_duration"] == 0]
assert zero_fwd_flow_duration.shape[0] == zero_fwd_flow_duration.loc[:, "total_fwd_packets"].sum()

NOTE: there are cases when flow_duration is > 0 but both bwd_flow_duration and fwd_flow_duration = 0. These are cases where at most one forward and backward packet

In [ ]:
# Sanity check for this case as well 
mismatched_durations = df[(df.loc[:, "flow_duration"] > 0) & (df.loc[:, "fwd_flow_duration"] == 0) & (df.loc[:, "bwd_flow_duration"] == 0)]
mismatched_durations.loc[:, ["total_fwd_packets", "total_bwd_packets", "flow_duration", "fwd_flow_duration", "bwd_flow_duration"]]

,total_fwd_packets,total_bwd_packets,flow_duration,fwd_flow_duration,bwd_flow_duration


In [ ]:
# Make these assertions for checking
if mismatched_durations.shape[0] > 0:
    assert mismatched_durations.loc[:, "total_fwd_packets"].max() == 1
    assert mismatched_durations.loc[:, "total_bwd_packets"].max() == 1

Converting timestamps to human-readable form

In [36]:
df_test = df.copy()

# Overall
df_test.loc[:, "first_timestamp"] = df.loc[:, "first_timestamp"].apply(lambda x: datetime.fromtimestamp(float(x), UTC).strftime("%Y-%m-%d %H:%M:%S.%f"))
df_test.loc[:, "last_timestamp"] = df.loc[:, "last_timestamp"].apply(lambda x: datetime.fromtimestamp(float(x), UTC).strftime("%Y-%m-%d %H:%M:%S.%f"))

# Forward 
df_test.loc[:, "first_timestamp_fwd"] = df.loc[:, "first_timestamp_fwd"].apply(lambda x: datetime.fromtimestamp(float(x), UTC).strftime("%Y-%m-%d %H:%M:%S.%f"))
df_test.loc[:, "last_timestamp_fwd"] = df.loc[:, "last_timestamp_fwd"].apply(lambda x: datetime.fromtimestamp(float(x), UTC).strftime("%Y-%m-%d %H:%M:%S.%f"))

# Backward 
df_test.loc[:, "first_timestamp_bwd_new"] = df.loc[:, "first_timestamp_bwd"].apply(lambda x: datetime.fromtimestamp(float(x), UTC).strftime("%Y-%m-%d %H:%M:%S.%f") if not pd.isna(x) else np.nan)
df_test.loc[:, "last_timestamp_bwd_new"] = df.loc[:, "last_timestamp_bwd"].apply(lambda x: datetime.fromtimestamp(float(x), UTC).strftime("%Y-%m-%d %H:%M:%S.%f") if not pd.isna(x) else np.nan)

Sanity check to make sure timestamp conversion preserves NaN values

In [ ]:
df_test[df_test.loc[:, "first_timestamp_bwd"].isna()].loc[:, ["first_timestamp_bwd_new", "first_timestamp_bwd"]].head()

,first_timestamp_bwd_new,first_timestamp_bwd
3284,NaN,NaN
4411,NaN,NaN


In [ ]:
df.columns

Index(['sport', 'src_ip', 'dst_port', 'dst_ip', 'protocol', 'sizes',
       'first_timestamp', 'last_timestamp', 'flow_duration', 'arrival_times',
       'total_fwd_packets', 'fwd_pkt_sizes', 'first_timestamp_fwd',
       'last_timestamp_fwd', 'arrival_times_fwd', 'total_bwd_packets',
       'bwd_pkt_sizes', 'first_timestamp_bwd', 'last_timestamp_bwd',
       'arrival_times_bwd', 'syn_flag_count', 'fin_flag_count',
       'rst_flag_count', 'psh_flag_count', 'ack_flag_count', 'urg_flag_count',
       'cwr_flag_count', 'ece_flag_count', 'total_size', 'avg_size',
       'std_size', 'total_fwd_pkt_size', 'avg_fwd_pkt_size',
       'std_fwd_pkt_size', 'total_bwd_pkt_size', 'avg_bwd_pkt_size',
       'std_bwd_pkt_size', 'fwd_flow_duration', 'bwd_flow_duration'],
      dtype='object')

Computing inter-arrival times and statistics

In [ ]:
import warnings

warnings.filterwarnings("ignore")

df_time = df_test.copy()
def find_diff(arrival_times):
    return [float(arrival_times[i+1] - arrival_times[i]) for i in range(len(arrival_times)-1)]

# Overall 
df_time.loc[:, "inter_arrival_times"] = df_time.loc[:, "arrival_times"].apply(find_diff)
df_time.loc[:, "inter_arrival_mean"] = df_time.loc[:, "inter_arrival_times"].apply(lambda x: np.mean(x))
df_time.loc[:, "inter_arrival_std"] = df_time.loc[: ,"inter_arrival_times"].apply(lambda x: np.std(x))

# Forward
df_time.loc[:, "inter_arrival_times_fwd"] = df_time.loc[:, "arrival_times_fwd"].apply(find_diff)
df_time.loc[:, "inter_arrival_mean_fwd"] = df_time.loc[:, "inter_arrival_times_fwd"].apply(lambda x: np.mean(x))
df_time.loc[:, "inter_arrival_std_fwd"] = df_time.loc[: ,"inter_arrival_times_fwd"].apply(lambda x: np.std(x))

# Backward
df_time.loc[:, "inter_arrival_times_bwd"] = df_time.loc[:, "arrival_times_bwd"].apply(find_diff)
df_time.loc[:, "inter_arrival_mean_bwd"] = df_time.loc[:, "inter_arrival_times_bwd"].apply(lambda x: np.mean(x))
df_time.loc[:, "inter_arrival_std_bwd"] = df_time.loc[: ,"inter_arrival_times_bwd"].apply(lambda x: np.std(x))

# df_time[df_time.loc[:, "arrival_times"].apply(lambda x: len(x) == 1)].loc[:, ["arrival_times", "inter_arrival_times", "inter_arrival_mean", "inter_arrival_std"]]

In [ ]:
df_time.loc[:, ["arrival_times", "inter_arrival_times", "inter_arrival_mean", "inter_arrival_std"]]

,arrival_times,inter_arrival_times,inter_arrival_mean,inter_arrival_std
0,"[1499453793.83595, 1499453793.836082, 14994537...","[0.000132, 0.000888, 4.9e-05, 7.9e-05, 0.01433...",0.702354,2.324233
1,"[1499453793.850928, 1499453793.851047, 1499453...","[0.000119, 0.000849, 4.8e-05, 8.9e-05, 0.00844...",22.735791,112.563018
2,"[1499453793.875876, 1499453793.876012, 1499453...","[0.000136, 0.000857, 4.8e-05, 6.4e-05, 1.6548,...",21.133950,101.011109
3,"[1499453796.193416, 1499453796.193514, 1499453...","[9.8e-05, 0.000612, 1.8e-05, 5.9e-05, 1.344956...",0.570284,1.626237
4,"[1499453797.195724, 1499453797.195832, 1499453...","[0.000108, 0.000704, 4e-06, 6e-05, 0.655491, 5...",0.493280,1.528730
...,...,...,...,...
8578,"[1499454970.599251, 1499454970.599333, 1499454...","[8.2e-05, 0.00074, 0.000361, 0.000146, 0.00110...",0.145719,0.455978
8579,"[1499454970.614938, 1499454970.614986, 1499454...","[4.8e-05, 0.000503, 0.00171, 0.00018, 0.000257...",0.132185,0.435178
8580,"[1499454970.630293, 1499454970.630338, 1499454...","[4.5e-05, 0.000719, 0.000262, 6.3e-05, 1.56841...",0.224838,0.548513
8581,"[1499454970.630335, 1499454970.630523, 1499454...","[0.000188, 0.00062, 0.000198, 6.6e-05, 1.56843...",0.143255,0.450682


## Features to keep 

In [ ]:
df_time.columns

Index(['sport', 'src_ip', 'dst_port', 'dst_ip', 'protocol', 'sizes',
       'first_timestamp', 'last_timestamp', 'flow_duration', 'arrival_times',
       'total_fwd_packets', 'fwd_pkt_sizes', 'first_timestamp_fwd',
       'last_timestamp_fwd', 'arrival_times_fwd', 'total_bwd_packets',
       'bwd_pkt_sizes', 'first_timestamp_bwd', 'last_timestamp_bwd',
       'arrival_times_bwd', 'syn_flag_count', 'fin_flag_count',
       'rst_flag_count', 'psh_flag_count', 'ack_flag_count', 'urg_flag_count',
       'cwr_flag_count', 'ece_flag_count', 'total_size', 'avg_size',
       'std_size', 'total_fwd_pkt_size', 'avg_fwd_pkt_size',
       'std_fwd_pkt_size', 'total_bwd_pkt_size', 'avg_bwd_pkt_size',
       'std_bwd_pkt_size', 'fwd_flow_duration', 'bwd_flow_duration',
       'first_timestamp_bwd_new', 'last_timestamp_bwd_new',
       'inter_arrival_times', 'inter_arrival_mean', 'inter_arrival_std',
       'inter_arrival_times_fwd', 'inter_arrival_mean_fwd',
       'inter_arrival_std_fwd', 'inter_ar

In [ ]:
general_features = ["sport", "src_ip", "dst_port", "dst_ip", "protocol"]
overall_features = ["first_timestamp", "last_timestamp", "flow_duration", "total_size", "avg_size", "std_size", "inter_arrival_mean", "inter_arrival_std"]
fwd_features = ["total_fwd_packets", "first_timestamp_fwd", "last_timestamp_fwd", "fwd_flow_duration", "total_fwd_pkt_size", "avg_fwd_pkt_size", "std_fwd_pkt_size", "inter_arrival_mean_fwd", "inter_arrival_std_fwd"]
bwd_features = ['total_bwd_packets', 'first_timestamp_bwd', 'last_timestamp_bwd', 'bwd_flow_duration', 'total_bwd_pkt_size', 'avg_bwd_pkt_size', 'std_bwd_pkt_size', 'inter_arrival_mean_bwd', 'inter_arrival_std_bwd']
flag_features = ['syn_flag_count', 'fin_flag_count', 'rst_flag_count', 'psh_flag_count', 'ack_flag_count', 'urg_flag_count','cwr_flag_count', 'ece_flag_count']

In [ ]:
features_to_keep = general_features + overall_features + fwd_features + bwd_features + flag_features
print(f"Total features kept: {len(features_to_keep)}")

Total features kept: 39


## Attaching label for model data

In [ ]:
df_final = df_time.copy()
df_final = df_final[features_to_keep]
df_final.loc[:, "label"] = "port_scan"

## Saving as csv

In [ ]:
df_final.to_csv("port_scan_check.csv", index=False)